In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
pivot_table = "../data/pivot_table.xlsx"
dfs = pd.read_excel(pivot_table, sheet_name=None)
dfs_pivot_original = dfs["Сводная ПП+Потребности"]
dfs_potrebnosti_orginal = dfs["Потребности"]
dfs_pp_original = dfs["ПП"]

In [3]:
dfs_pivot_original.columns

Index(['Потребитель', 'Отдел', 'Ответственный', 'Установка', 'Материал ПП',
       'Материал потребитель', 'Товар', 'Производитель', 'Сумма ПП',
       'Сумма Потребности', 'Срок поставки', 'Вероятность следующей продажи',
       'Конкуренты', 'Ситуация', 'Решение/следующее действие', 'Дата контроля',
       'Примечание'],
      dtype='object')

In [4]:
dfs_potrebnosti_orginal.columns

Index(['Потребитель', 'Материал', 'Сумма потребности', 'Срок поставки',
       'Примечание', 'Отдел', 'Ответственный'],
      dtype='object')

In [21]:
main_col = ["Потребитель", "Отдел", 'Ответственный']
col_compare = ["Материал потребитель", "Материал"] 
col4match_potreb_2 = ["Потребитель", "Отдел" ,"Ответственный",'Материал','Примечание']
col4match_pivot_1= ["Потребитель", "Отдел" ,"Ответственный", "Материал ПП", 'Установка', 'Товар', 'Производитель']
col4match_pivot_2 = ["Потребитель", "Отдел" ,"Ответственный","Материал потребитель",'Примечание']
col4match_pp = ["Потребитель", "Отдел" ,"МП", "Материал, применение", 'Установка', 'Товар', 'Производитель']

## data preprocessing

In [10]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Загрузка стоп-слов и других ресурсов NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

# Функция для замены знаков препинания пробелами
def remove_punctuation(text):
    if isinstance(text, str):  # Проверяем, что text это строка
        s_cleaned = re.sub(r'^\s+|\s+$', '', text) # пробелы в начале и в конце строки 
        return re.sub(r'[^\w\s]', ' ', s_cleaned)  # Заменяем все символы, кроме букв, цифр и пробелов, на пробелы
    return text

# Функция для приведения текста к нижнему регистру
def to_lowercase(text):
    if isinstance(text, str):  # Проверяем, что text это строка
        return text.lower()
    return text

# Функция для удаления цифр
def remove_numbers(text):
    if isinstance(text, str):  # Проверяем, что text это строка
        return re.sub(r'\d+', '', text)
    return text


# Функция для удаления стоп-слов (предлогов, союзов и т.д.)
def remove_stopwords(text):
    if isinstance(text, str):  # Проверяем, что text это строка
        stop_words = set(stopwords.words('russian'))  # Можно использовать 'english' для английского текста
        words = nltk.word_tokenize(text)
        return ' '.join([word for word in words if word not in stop_words])
    return text

# Функция для стемминга
stemmer = SnowballStemmer("russian")

def stem_text(text):
    if isinstance(text, str):  # Проверяем, что text это строка
        words = nltk.word_tokenize(text)
        return ' '.join([stemmer.stem(word) for word in words])
    return text

def remove_nan_rows(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Столбец {column_name} не найден в DataFrame")
    # Удалить строки с NaN в указанном столбце
    df_cleaned = df.dropna(subset=[column_name])
    return df_cleaned

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/wormsin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/wormsin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/wormsin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [11]:
def clean_text_column(df, column, oper):
    if oper[5] == 1:
        df = remove_nan_rows(df, column)
    if oper[0] == 1:
        df[column] = df[column].apply(remove_punctuation)
    if oper[1] == 1:
        df[column] = df[column].apply(to_lowercase)
    if oper[2] == 1:
        df[column] = df[column].apply(remove_numbers)
    if oper[3] == 1:
        df[column] = df[column].apply(stem_text)
    if oper[4] == 1:
        df[column] = df[column].apply(remove_stopwords)
    return df

In [12]:
list_of_strings = ["remove_punctuation", "to_lowercase", "remove_numbers", "stem_text", "remove_stopwords", "remove_nan_rows"]
dict_from_list = {i: s for i, s in enumerate(list_of_strings)}
dict_from_list

{0: 'remove_punctuation',
 1: 'to_lowercase',
 2: 'remove_numbers',
 3: 'stem_text',
 4: 'remove_stopwords',
 5: 'remove_nan_rows'}

In [13]:
def data_preprocess_pivot(dfs):
    dfs =clean_text_column(dfs, "Материал ПП", [1, 1, 0, 1, 1, 0])
    dfs =clean_text_column(dfs, "Товар", [1, 1, 0, 0, 1, 0])
    dfs =clean_text_column(dfs, "Установка", [1, 1, 0, 0, 0, 0])
    dfs =clean_text_column(dfs, "Производитель", [1, 1, 0, 0, 0, 0])
    
    dfs = clean_text_column(dfs, "Материал потребитель", [1, 1, 0, 1, 1, 1])
    dfs = clean_text_column(dfs, "Примечание", [1, 1, 0, 1, 1, 0])
    
    dfs = clean_text_column(dfs, "Потребитель", [1, 1, 0, 0, 0, 1])
    dfs = clean_text_column(dfs, "Отдел", [1, 1, 0, 0, 0, 0])
    dfs = clean_text_column(dfs, "Ответственный", [1, 1, 0, 0, 0, 0])
    
    return dfs
    
def data_preprocess_potreb(dfs):
    dfs = clean_text_column(dfs, "Потребитель", [1, 1, 0, 0, 0, 1])
    dfs = clean_text_column(dfs, "Отдел", [1, 1, 0, 0, 0, 0])
    dfs = clean_text_column(dfs, "Ответственный", [1, 1, 0, 0, 0, 0])
    dfs = clean_text_column(dfs, "Материал", [1, 1, 0, 1, 1, 1])
    return dfs

def data_preprocess_pp(dfs):
    dfs = clean_text_column(dfs, "Потребитель", [1, 1, 0, 0, 0, 1])
    dfs = clean_text_column(dfs, "Отдел", [1, 1, 0, 0, 0, 0])
    dfs = clean_text_column(dfs, "МП", [1, 1, 0, 0, 0, 0])
    dfs = clean_text_column(dfs, "Материал, применение", [1, 1, 0, 1, 1, 1])
    dfs =clean_text_column(dfs, "Товар", [1, 1, 0, 0, 1, 0])
    dfs =clean_text_column(dfs, "Установка", [1, 1, 0, 0, 0, 0])
    dfs =clean_text_column(dfs, "Производитель", [1, 1, 0, 0, 0, 0])
    return dfs

## create dataset

In [ ]:
dfs_pivot = data_preprocess_pivot(dfs_pivot_original.copy())
dfs_potrebnosti = data_preprocess_potreb(dfs_potrebnosti_orginal.copy())
dfs_pivot = dfs_pivot.reset_index()
dfs_potrebnosti = dfs_potrebnosti.reset_index()

In [42]:
data = {
    'pp': [],
    'positive_potreb': [],
    'negative_potreb': []
}

df_dataset = pd.DataFrame(data)

In [43]:
from rapidfuzz import fuzz
import random
с = 0
for i, pp_row in dfs_pivot.iterrows():
    pp = ' '.join(np.array(pp_row[col4match_pivot_1].tolist()).astype(str))
    positive_potreb = " ".join(np.array(pp_row[col4match_pivot_2].tolist()).astype(str))
    
    matching_rows = dfs_potrebnosti[dfs_potrebnosti["Потребитель"].apply(lambda x: fuzz.ratio(x, pp_row["Потребитель"]) > 60)]
    matching_rows = matching_rows[matching_rows["Отдел"].apply(lambda x: fuzz.ratio(x, pp_row["Отдел"]) > 60)]
    matching_rows = matching_rows[matching_rows['Материал']!= pp_row["Материал потребитель"]]
    if len(matching_rows)!=0:
        rand_indx = random.choice(matching_rows.index)
        row = matching_rows.loc[rand_indx]
    else:
        rand_indx = random.choice(dfs_potrebnosti.index)
        row = dfs_potrebnosti.loc[rand_indx]
        с+=1
    
    negative_potreb = " ".join(np.array(row[col4match_potreb_2].tolist()).astype(str))
    new_index = len(df_dataset)
    df_dataset.loc[new_index] = [pp, positive_potreb, negative_potreb]
        

In [47]:
len(df_dataset)

100

In [50]:
df_dataset = df_dataset.replace('nan', '', regex=True)

In [51]:
df_dataset.to_csv('dataset.csv', index=False)

## triplet loss ruBert model 

In [52]:
from transformers import AutoTokenizer, AutoModel
import torch

class TripletBERT(torch.nn.Module):
    def __init__(self, model_name="DeepPavlov/rubert-base-cased"):
        super(TripletBERT, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # Используем выход скрытого слоя на уровне [CLS] токена как эмбеддинг
        return output.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = TripletBERT()


/home/wormsin/miniconda3/envs/file_clean/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. in

In [53]:
for name, param in model.bert.named_parameters():
    if "encoder.layer" in name and int(name.split('.')[2]) < 8:
        param.requires_grad = False

In [63]:
from torch.utils.data import Dataset, DataLoader

class TripletTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Получаем строки
        anchor = str(self.data.iloc[index, 0])
        positive = str(self.data.iloc[index, 1])
        negative = str(self.data.iloc[index, 2])

        # Токенизация Anchor
        anchor_inputs = self.tokenizer.encode_plus(
            anchor,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        
        # Токенизация Positive
        positive_inputs = self.tokenizer.encode_plus(
            positive,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        
        # Токенизация Negative
        negative_inputs = self.tokenizer.encode_plus(
            negative,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )

        return {
            'anchor_input_ids': anchor_inputs['input_ids'].squeeze(0),
            'anchor_attention_mask': anchor_inputs['attention_mask'].squeeze(0),
            'positive_input_ids': positive_inputs['input_ids'].squeeze(0),
            'positive_attention_mask': positive_inputs['attention_mask'].squeeze(0),
            'negative_input_ids': negative_inputs['input_ids'].squeeze(0),
            'negative_attention_mask': negative_inputs['attention_mask'].squeeze(0),
        }


In [ ]:
df = pd.read_csv("dataset.csv") 
print(df.head())

In [66]:
dataset = TripletTextDataset(df, tokenizer, max_length=64)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [65]:
import torch.nn.functional as F

def triplet_loss(anchor, positive, negative, margin=1.0):
    # Косинусная близость
    pos_distance = F.cosine_similarity(anchor, positive)
    neg_distance = F.cosine_similarity(anchor, negative)

    # Triplet Loss
    loss = torch.mean(F.relu(neg_distance - pos_distance + margin))
    return loss


In [74]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
device = 'cuda'
model.to(device)
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        anchor_input_ids = batch['anchor_input_ids'].to(device)
        anchor_attention_mask = batch['anchor_attention_mask'].to(device)
        positive_input_ids = batch['positive_input_ids'].to(device)
        positive_attention_mask = batch['positive_attention_mask'].to(device)
        negative_input_ids = batch['negative_input_ids'].to(device)
        negative_attention_mask = batch['negative_attention_mask'].to(device)

        # Получаем эмбеддинги для Anchor, Positive и Negative
        anchor_emb = model(anchor_input_ids, anchor_attention_mask)
        positive_emb = model(positive_input_ids, positive_attention_mask)
        negative_emb = model(negative_input_ids, negative_attention_mask)

        # Вычисляем Triplet Loss
        loss = triplet_loss(anchor_emb, positive_emb, negative_emb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")


Epoch 1/200, Loss: 0.9062754958868027
Epoch 2/200, Loss: 0.821341797709465
Epoch 3/200, Loss: 0.7497813701629639
Epoch 4/200, Loss: 0.6198317110538483
Epoch 5/200, Loss: 0.5784109234809875
Epoch 6/200, Loss: 0.5761526823043823
Epoch 7/200, Loss: 0.4265815205872059
Epoch 8/200, Loss: 0.5029339417815208
Epoch 9/200, Loss: 0.3521309345960617
Epoch 10/200, Loss: 0.35863395780324936
Epoch 11/200, Loss: 0.3530420958995819
Epoch 12/200, Loss: 0.267801932990551
Epoch 13/200, Loss: 0.2833317443728447
Epoch 14/200, Loss: 0.25822220370173454
Epoch 15/200, Loss: 0.2816007360816002
Epoch 16/200, Loss: 0.17239025980234146
Epoch 17/200, Loss: 0.1647677905857563
Epoch 18/200, Loss: 0.1715560145676136
Epoch 19/200, Loss: 0.16969040036201477
Epoch 20/200, Loss: 0.1473044566810131
Epoch 21/200, Loss: 0.14785170927643776
Epoch 22/200, Loss: 0.11724194884300232
Epoch 23/200, Loss: 0.09736148081719875
Epoch 24/200, Loss: 0.10676917806267738
Epoch 25/200, Loss: 0.09608535282313824
Epoch 26/200, Loss: 0.18496

In [76]:
torch.save(model.bert, 'bert_model/200_epochs.pth')

## предобучение модели на словаре

In [13]:
import pandas as pd

dfs_pp = dfs_pp_original[col4match_pp]
dfs_potrebnosti = dfs_potrebnosti_orginal[col4match_potreb_2]


In [ ]:
dfs_pp['combined'] = dfs_pp[col4match_pp].apply(lambda row: ' [SEP] '.join(row.values.astype(str)), axis=1)
dfs_potrebnosti['combined'] = dfs_potrebnosti[col4match_potreb_2].apply(lambda row: ' [SEP] '.join(row.values.astype(str)), axis=1)


In [ ]:
dfs_pp['combined'] = dfs_pp['combined'].replace('nan', '', regex=True)
dfs_potrebnosti['combined'] = dfs_potrebnosti['combined'].replace('nan', '', regex=True)

In [23]:
len(dfs_potrebnosti['combined'])

26127

In [32]:
data = {
    'combined':[]

}
df = pd.DataFrame(data)
df['combined'] = pd.concat((dfs_pp['combined'], dfs_potrebnosti['combined']), axis=0)

In [33]:
df

,combined
0,"РН-Сызранский НПЗ, АО [SEP] КиА НП [SEP] Жилен..."
1,ЛУКОЙЛ-Пермнефтеоргсинтез [SEP] КиА НП [SEP] Ш...
2,ЛУКОЙЛ-Пермнефтеоргсинтез [SEP] КиА НП [SEP] Ш...
3,ЛУКОЙЛ-Пермнефтеоргсинтез [SEP] КиА НП [SEP] Ш...
4,ЛУКОЙЛ-Пермнефтеоргсинтез [SEP] КиА НП [SEP] Ш...
...,...
26122,Башнефть-Новойл [SEP] КиА НП [SEP] Жиленко [SE...
26123,Башнефть-Новойл [SEP] КиА НП [SEP] Жиленко [SE...
26124,Башнефть-Новойл [SEP] КиА НП [SEP] Жиленко [SE...
26125,Башнефть-Новойл [SEP] КиА НП [SEP] Жиленко [SE...


In [34]:
# Сохраняем объединенные строки в txt-файл
df['combined'].to_csv('data_for_finetuning.txt', index=False, header=False)


In [1]:
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset

# Загружаем предобученную модель и токенайзер
model = BertForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

# Подготавливаем датасет из вашего словаря
dataset = load_dataset('text', data_files={'train': 'data_for_finetuning.txt'})
#dataset = dataset['train'].map(lambda examples: tokenizer(examples['text'], truncation=True, padding='max_length', max_length=64), batched=True)

# Шаг 3: Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Шаг 4: Использование DataCollator для автоматической маскировки токенов
# DataCollatorForLanguageModeling автоматически замаскирует часть токенов
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=True, 
    mlm_probability=0.15  # 15% токенов будут замаскированы
)

# Шаг 5: Аргументы для тренировки
training_args = TrainingArguments(
    output_dir="bert_finetuned",  # Директория для сохранения модели
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,  # Оставляем только потери (loss) для упрощения вывода
)

# Шаг 6: Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    data_collator=data_collator,  # Для автоматической маскировки токенов
)

# Дообучаем модель на вашем словаре
trainer.train()

# Сохраняем дообученную модель
model.save_pretrained('bert_finetuned')
tokenizer.save_pretrained('bert_finetuned')


/home/wormsin/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wormsin/miniconda3/envs/file_clean/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
 10%|█         | 500/4959 [03:37<31:32,  2.36it/s]

{'loss': 1.3704, 'grad_norm': 8.495681762695312, 'learning_rate': 4.495866102036701e-05, 'epoch': 0.3}


 20%|██        | 1000/4959 [07:12<28:39,  2.30it/s]

{'loss': 0.9021, 'grad_norm': 9.423467636108398, 'learning_rate': 3.991732204073402e-05, 'epoch': 0.6}


 30%|███       | 1500/4959 [10:47<24:30,  2.35it/s]

...

{'loss': 0.4594, 'grad_norm': 10.365859031677246, 'learning_rate': 9.669288162936076e-06, 'epoch': 2.42}


 91%|█████████ | 4500/4959 [32:17<03:16,  2.33it/s]

{'loss': 0.4391, 'grad_norm': 8.54740047454834, 'learning_rate': 4.627949183303086e-06, 'epoch': 2.72}


100%|██████████| 4959/4959 [35:34<00:00,  2.42it/s]

ValueError: You are trying to save a non contiguous tensor: `bert.encoder.layer.0.attention.self.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

In [7]:
import torch
torch.save(model.bert, 'bert_finetuned/3_epochs_bert.pth')

In [6]:
tokenizer.save_pretrained('bert_finetuned/tokenizer')

('bert_finetuned/tokenizer/tokenizer_config.json',
 'bert_finetuned/tokenizer/special_tokens_map.json',
 'bert_finetuned/tokenizer/vocab.txt',
 'bert_finetuned/tokenizer/added_tokens.json',
 'bert_finetuned/tokenizer/tokenizer.json')

## Faiss

In [ ]:
import pandas as pd
pivot_table = "../data/pivot_table.xlsx"
dfs = pd.read_excel(pivot_table, sheet_name=None)
dfs_pp_original = dfs["ПП"]
dfs_potrebnosti_orginal = dfs["Потребности"]
dfs_potrebnosti = data_preprocess_potreb(dfs_potrebnosti_orginal.copy())
dfs_potrebnosti = dfs_potrebnosti.reset_index()
dfs_pp = data_preprocess_pp(dfs_pp_original.copy())
dfs_pp = dfs_pp.reset_index()

In [15]:
from transformers import AutoTokenizer, AutoModel
import torch 

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = torch.load('bert_finetuned/3_epochs_bert.pth').to('cpu')

/home/wormsin/miniconda3/envs/file_clean/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/tmp/ipykernel_6540/2228023973.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed 

In [16]:
data = {
    'string':[]
}

df_vec1= pd.DataFrame(data)
df_vec2= pd.DataFrame(data)

In [17]:
def create_vec_data(col4match, max_length, dfs, df):
    for indx, row in dfs.iterrows():
        anchor = ' '.join(np.array(row[col4match].tolist()).astype(str))
        inputs = tokenizer.encode_plus(
                    anchor,
                    add_special_tokens=True,
                    max_length=max_length,
                    truncation=True,
                    return_tensors="pt"
                )
        with torch.no_grad():
            outputs = model(**inputs)
        df.loc[indx] = [outputs.last_hidden_state[:, 0, :].numpy()]

In [19]:
import numpy as np
col4match = ["Потребитель", "Отдел" ,"МП", "Материал, применение", 'Установка', 'Товар', 'Производитель']
create_vec_data(col4match, max_length=64, dfs=dfs_pp, df=df_vec1)
#create_vec_data(col4match_potreb_2, max_length=64, dfs=dfs_potrebnosti, df=df_vec2)

In [22]:
create_vec_data(col4match_potreb_2, max_length=64, dfs=dfs_potrebnosti, df=df_vec2)

In [23]:
vec1 = np.vstack(df_vec1['string'].to_numpy())

In [24]:
vec2 = np.vstack(df_vec2['string'].to_numpy())

In [27]:
import faiss
import numpy as np

# Примерные данные (вместо этого используйте ваши векторы)
# 300 векторов с классами (например, 128-мерные)
table_a_vectors = vec1

# 30,000 векторов без классов
table_b_vectors = vec2

index = faiss.IndexFlatIP(768)

# Добавляем нормализованные вектора
faiss.normalize_L2(table_a_vectors)
faiss.normalize_L2(table_b_vectors)
index.add(table_b_vectors)
# Создание индекса
#index = faiss.IndexFlatL2(768)  # Индекс с L2 расстоянием
#index.add(table_b_vectors)  # Добавление векторов из первой таблицы

# Поиск ближайших соседей для векторов из второй таблицы
k = 5  # Количество ближайших соседей
distances, indices = index.search(table_a_vectors, k)

# Индексы ближайших соседей и их расстояния
print("Индексы ближайших соседей:", indices)
print("Расстояния до ближайших соседей:", distances)



Индексы ближайших соседей: [[20945 14863 14409 23565 13909]
 [14483 17272 25515  1029 21123]
 [12691 10591 18329 10665 19571]
 ...
 [13578 24346 13118 10604 18149]
 [   50 23045 23443  2247 21088]
 [22095 18741 25515 24494 18485]]
Расстояния до ближайших соседей: [[0.91479737 0.9108299  0.9082465  0.90682155 0.9062322 ]
 [0.94147086 0.9353296  0.9351766  0.9349063  0.93440205]
 [0.9170652  0.9094411  0.9078841  0.9075177  0.9072263 ]
 ...
 [0.7421916  0.72082764 0.71973836 0.71924835 0.71888053]
 [0.9601269  0.95820415 0.9561344  0.95613277 0.95416963]
 [0.94834495 0.94444263 0.9432762  0.9429506  0.9414692 ]]


In [ ]:
for i in range(len(indices)):
    indice = indices[i][indices[i]<10000]
    for indx in indice:
        print('Строка ПП: \n')
        print(dfs_pp.loc[i][col4match])
        print('Строки ПОтребности: \n')
        print(dfs_potrebnosti.loc[indx][col4match_potreb_2])
        

In [28]:
# Фильтрация результатов
matches = []
for i in range(len(indices)):
    matches.append((i, indices[i][indices[i]>0.5]))  # Сохраните индекс строки из table_b и соответствующий индекс из table_a

# Сохранение результатов
matches_df = pd.DataFrame(matches, columns=['table_a_index', 'table_b_index'])
matches_df.to_csv('matches_finetuned_cos.csv', index=False)

## Анализ результатов

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [30]:
pivot_table = "../data/pivot_table.xlsx"
dfs = pd.read_excel(pivot_table, sheet_name=None)
dfs_pivot_original = dfs["Сводная ПП+Потребности"]
dfs_potrebnosti_orginal = dfs["Потребности"]
dfs_pp_original = dfs["ПП"]

In [ ]:
dfs_potrebnosti = data_preprocess_potreb(dfs_potrebnosti_orginal.copy())
dfs_potrebnosti = dfs_potrebnosti.reset_index()
dfs_pp = data_preprocess_pp(dfs_pp_original.copy())
dfs_pp = dfs_pp.reset_index()
dfs_pivot = data_preprocess_pivot(dfs_pivot_original.copy())

In [97]:
matches = pd.read_csv('matches.csv')

In [98]:
matches

,table_a_index,table_b_index
0,0,[]
1,1,[]
2,2,[ 354 9719 344]
3,3,[]
4,4,[ 60 9745]
...,...,...
266,266,[2475 32]
267,267,[2475]
268,268,[]
269,269,[9713]


In [ ]:
import ast
manual  = []
res = 0
for i, row in dfs_pivot.iterrows():
    pp_row = dfs_pp[(dfs_pp["Потребитель"] == row["Потребитель"]) &
                    (dfs_pp["Отдел"] == row["Отдел"]) &
                    (dfs_pp["МП"] == row["Ответственный"]) &
                    (dfs_pp["Материал, применение"] == row["Материал ПП"]) &
                    (dfs_pp["Установка"] == row["Установка"]) &
                    (dfs_pp["Товар"] == row["Товар"])].index
    if len(pp_row) ==1:
        indx = pp_row[0]
        matches_indices = list(map(int, matches.loc[indx]["table_b_index"].strip('[]').split()))
        print(f"New: {row["Материал потребитель"]}")
        for k in matches_indices:
            print(dfs_potrebnosti.loc[k]['Материал'])
            if dfs_potrebnosti.loc[k]['Материал'] == row["Материал потребитель"]:
                res+=1
    else:
        manual+=pp_row.tolist()

In [100]:
res

7